# REINFORCE

---

In this notebook, we will train REINFORCE with OpenAI Gym's Cartpole environment.

### 1. Import the Necessary Packages

In [1]:
import gym
gym.logger.set_level(40) # suppress warnings (please remove if gives error)
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import torch
torch.manual_seed(0) # set random seed
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

!python -m pip install pyvirtualdisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

is_ipython = 'inline' in plt.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

  Running setup.py bdist_wheel for pyvirtualdisplay ... done
  Stored in directory: /root/.cache/pip/wheels/d1/8c/16/1c64227974ae29c687e4cc30fd691d5c0fd40f54446dde99da
  Running setup.py bdist_wheel for EasyProcess ... done
  Stored in directory: /root/.cache/pip/wheels/b4/c6/e3/c163b04029d8fccfd54b809802640c1af587a01be8d7a04e1a
Successfully built pyvirtualdisplay EasyProcess
You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### 2. Define the Architecture of the Policy

In [2]:
env = gym.make('CartPole-v0')
env.seed(0)
print('observation space:', env.observation_space)
print('action space:', env.action_space)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Policy(nn.Module):
    def __init__(self, s_size=4, h_size=16, a_size=2):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

observation space: Box(4,)
action space: Discrete(2)


### 3. Train the Agent with REINFORCE

In [4]:
policy = Policy().to(device)
optimizer = optim.Adam(policy.parameters(), lr=1e-2)

def reinforce(n_episodes=1000, max_t=1000, gamma=1.0, print_every=100):
    scores_deque = deque(maxlen=100)
    scores = []
    for i_episode in range(1, n_episodes+1):
        saved_log_probs = []
        rewards = []
        state = env.reset()
        for t in range(max_t):
            action, log_prob = policy.act(state)
            print(state)
            saved_log_probs.append(log_prob)
            state, reward, done, _ = env.step(action)
            rewards.append(reward)
            if done:
                break 
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))
        
        discounts = [gamma**i for i in range(len(rewards)+1)]
        R = sum([a*b for a,b in zip(discounts, rewards)])
        
        policy_loss = []
        for log_prob in saved_log_probs:
            policy_loss.append(-log_prob * R)
        policy_loss = torch.cat(policy_loss).sum()
        
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        if np.mean(scores_deque)>=195.0:
            print('Environment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
            break
        
    return scores
    
scores = reinforce()

[-0.01032251  0.04644526  0.00232086 -0.04521839]
[-0.00939361 -0.1487099   0.00141649  0.24819588]
[-0.0123678  -0.34385205  0.00638041  0.54132526]
[-0.01924485 -0.5390631   0.01720692  0.8360117 ]
[-0.03002611 -0.34418036  0.03392715  0.54878945]
[-0.03690971 -0.14955102  0.04490294  0.26698612]
[-0.03990074 -0.3452841   0.05024266  0.57348683]
[-0.04680642 -0.15090125  0.0617124   0.29704561]
[-0.04982444 -0.34684616  0.06765331  0.60853541]
[-0.05676137 -0.54284544  0.07982402  0.92173667]
[-0.06761827 -0.73894999  0.09825875  1.23840018]
[-0.08239727 -0.5452178   0.12302675  0.97804521]
[-0.09330163 -0.35194094  0.14258766  0.72639986]
[-0.10034045 -0.54871629  0.15711566  1.06034632]
[-0.11131477 -0.35598434  0.17832258  0.82081149]
[-0.11843446 -0.55303958  0.19473881  1.1638566 ]
[ 0.04123038 -0.02819456  0.0157849  -0.00548869]
[ 0.04066649  0.1666975   0.01567513 -0.29314979]
[ 0.04400044  0.3615925   0.00981213 -0.58084807]
[ 0.05123229  0.16633444 -0.00180483 -0.28509036]


[ 0.01378226 -0.21139155 -0.04000177  0.29942026]
[ 0.00955442 -0.40592117 -0.03401337  0.57922387]
[ 0.001436   -0.21033947 -0.02242889  0.27602301]
[-0.00277079 -0.01490482 -0.01690843 -0.02364877]
[-0.00306889  0.18045548 -0.01738141 -0.32161818]
[  5.40224346e-04   3.75820582e-01  -2.38137710e-02  -6.19731442e-01]
[ 0.00805664  0.18103918 -0.0362084  -0.33464277]
[ 0.01167742 -0.01354924 -0.04290126 -0.0535944 ]
[ 0.01140643 -0.20803062 -0.04397314  0.22525015]
[ 0.00724582 -0.01230869 -0.03946814 -0.08097288]
[ 0.00699965 -0.20684329 -0.0410876   0.19900115]
[ 0.00286278 -0.40135422 -0.03710757  0.47844495]
[-0.0051643  -0.59593317 -0.02753868  0.75920533]
[-0.01708296 -0.79066506 -0.01235457  1.04309706]
[-0.03289627 -0.59538124  0.00850737  0.74656153]
[-0.04480389 -0.79061953  0.0234386   1.04190951]
[-0.06061628 -0.59581661  0.04427679  0.75667571]
[-0.07253261 -0.40133198  0.05941031  0.47824779]
[-0.08055925 -0.20709691  0.06897526  0.20486557]
[-0.08470119 -0.01302566  0.07

[-0.06742733 -0.39008236  0.08079602  0.6571031 ]
[-0.07522898 -0.58623053  0.09393808  0.97409371]
[-0.08695359 -0.39248577  0.11341995  0.71233509]
[-0.09480331 -0.58898023  0.12766665  1.03845703]
[-0.10658291 -0.78554617  0.14843579  1.36833849]
[-0.12229384 -0.59256057  0.17580256  1.12552335]
[-0.13414505 -0.40012282  0.19831303  0.89273607]
[-0.0069256  -0.03365656  0.01322477 -0.0125272 ]
[-0.00759873  0.16127326  0.01297422 -0.30100839]
[-0.00437327 -0.03403118  0.00695405 -0.00426207]
[-0.00505389 -0.22925217  0.00686881  0.29060681]
[ -9.63893548e-03  -3.42288350e-02   1.26809491e-02   9.81211327e-05]
[-0.01032351  0.16070898  0.01268291 -0.28855699]
[-0.00710933 -0.03459152  0.00691177  0.00809887]
[-0.00780116 -0.22981191  0.00707375  0.3029545 ]
[-0.0123974  -0.03479148  0.01313284  0.01251085]
[-0.01309323  0.16013969  0.01338306 -0.27599975]
[-0.00989044  0.35506817  0.00786306 -0.56443179]
[-0.00278907  0.55007892 -0.00342557 -0.85462715]
[ 0.00821251  0.74524739 -0.02

[-0.01727721 -0.23595014  0.10431824  0.5015082 ]
[-0.02199621 -0.43237594  0.11434841  0.82515916]
[-0.03064373 -0.6288606   0.13085159  1.1515066 ]
[-0.04322094 -0.82542415  0.15388172  1.48219202]
[-0.05972943 -0.63247758  0.18352556  1.24125496]
[-0.07237898 -0.8294186   0.20835066  1.5853596 ]
[-0.04835337 -0.03649855  0.02275268 -0.04876648]
[-0.04908334  0.15828988  0.02177735 -0.33418479]
[-0.04591754  0.35309523  0.01509365 -0.61992149]
[-0.03885563  0.54800316  0.00269522 -0.90781271]
[-0.02789557  0.35284482 -0.01546103 -0.61428388]
[-0.02083867  0.54817936 -0.02774671 -0.91179604]
[-0.00987509  0.3534436  -0.04598263 -0.62796132]
[-0.00280622  0.15899256 -0.05854186 -0.35010719]
[  3.73635873e-04   3.54896101e-01  -6.55440007e-02  -6.60660579e-01]
[ 0.00747156  0.16074451 -0.07875721 -0.38931467]
[ 0.01068645  0.35689082 -0.08654351 -0.70575282]
[ 0.01782426  0.55309848 -0.10065856 -1.02437487]
[ 0.02888623  0.35945059 -0.12114606 -0.76491702]
[ 0.03607525  0.16618665 -0.13

[ 0.03859317 -0.15088383 -0.02474534  0.2223062 ]
[ 0.0355755   0.04458291 -0.02029922 -0.07807854]
[ 0.03646715 -0.15024225 -0.02186079  0.20813143]
[ 0.03346231  0.04518536 -0.01769816 -0.0913665 ]
[ 0.03436602  0.24055646 -0.01952549 -0.38958023]
[ 0.03917714  0.43595002 -0.02731709 -0.68835484]
[ 0.04789615  0.63144025 -0.04108419 -0.989511  ]
[ 0.06052495  0.43689162 -0.06087441 -0.71000966]
[ 0.06926278  0.63280147 -0.0750746  -1.02121614]
[ 0.08191881  0.43875572 -0.09549893 -0.7530186 ]
[ 0.09069393  0.24507116 -0.1105593  -0.49184959]
[ 0.09559535  0.05166813 -0.12039629 -0.23595452]
[ 0.09662871  0.24828605 -0.12511538 -0.56405752]
[ 0.10159443  0.44492088 -0.13639653 -0.89339223]
[ 0.11049285  0.64160283 -0.15426438 -1.22565145]
[ 0.12332491  0.44876626 -0.1787774  -0.9850056 ]
[ 0.13230023  0.64577369 -0.19847752 -1.32808787]
[ 0.01714605 -0.03015641  0.02511599 -0.03477114]
[ 0.01654293 -0.22562936  0.02442057  0.26572913]
[ 0.01203034 -0.42109119  0.02973515  0.56601339]


...

[ 0.07041815  0.41027799 -0.06418311 -0.64105272]
[ 0.07862371  0.60623319 -0.07700416 -0.95323719]
[ 0.09074838  0.41222704 -0.0960689  -0.68570719]
[ 0.09899292  0.60854213 -0.10978305 -1.00702282]
[ 0.11116376  0.41504357 -0.1299235  -0.75073373]
[ 0.11946463  0.22192996 -0.14493818 -0.50159484]
[ 0.12390323  0.0291165  -0.15497008 -0.25786996]
[ 0.12448556 -0.16349262 -0.16012747 -0.01779764]
[ 0.12121571 -0.35599912 -0.16048343  0.22039144]
[ 0.11409572 -0.54850671 -0.1560756   0.45846243]
[ 0.10312559 -0.35156265 -0.14690635  0.12093699]
[ 0.09609434 -0.1546748  -0.14448761 -0.21424895]
[ 0.09300084  0.04218552 -0.14877259 -0.54879415]
[ 0.09384455 -0.15056784 -0.15974847 -0.30643588]
[ 0.09083319 -0.34309566 -0.16587719 -0.06808779]
[ 0.08397128 -0.53549901 -0.16723895  0.16801125]
[ 0.0732613  -0.33842722 -0.16387872 -0.17241152]
[ 0.06649276 -0.5308707  -0.16732695  0.06442116]
[ 0.05587534 -0.33379424 -0.16603853 -0.2760297 ]
[ 0.04919946 -0.52620618 -0.17155912 -0.03997242]


[ 0.02490342 -0.0188235  -0.08082077 -0.10552434]
[ 0.02452695  0.17735806 -0.08293125 -0.42257156]
[ 0.02807411 -0.01649708 -0.09138268 -0.15714207]
[ 0.02774417 -0.21019993 -0.09452553  0.10537073]
[ 0.02354017 -0.01385941 -0.09241811 -0.21557443]
[ 0.02326298 -0.20754696 -0.0967296   0.04658479]
[ 0.01911204 -0.01118063 -0.0957979  -0.27498196]
[ 0.01888843  0.18516831 -0.10129754 -0.59627648]
[ 0.02259179 -0.00840104 -0.11322307 -0.33714143]
[ 0.02242377 -0.20174499 -0.1199659  -0.08219895]
[ 0.01838887 -0.00512586 -0.12160988 -0.41019335]
[ 0.01828636  0.19149138 -0.12981375 -0.73860631]
[ 0.02211618  0.38814418 -0.14458587 -1.06916171]
[ 0.02987907  0.19519943 -0.16596911 -0.82512475]
[ 0.03378306  0.00268897 -0.1824716  -0.58889829]
[ 0.03383684 -0.18947218 -0.19424957 -0.35879134]
[ 0.03004739 -0.38137889 -0.2014254  -0.13309602]
[ 0.02241981 -0.57313147 -0.20408732  0.08989825]
[ 0.01095718 -0.76483253 -0.20228935  0.31189609]
[-0.00433947 -0.95658456 -0.19605143  0.53459089]


[-0.10634855  0.12421853  0.16291118  0.23527597]
[-0.10386418  0.31668351  0.1676167  -0.00191464]
[-0.09753051  0.50905459  0.1675784  -0.2373783 ]
[-0.08734941  0.70143551  0.16283084 -0.47286947]
[-0.0733207   0.50443358  0.15337345 -0.13361439]
[-0.06323203  0.30748536  0.15070116  0.20325528]
[-0.05708232  0.50016689  0.15476627 -0.03835262]
[-0.04707899  0.69276989  0.15399921 -0.27848415]
[-0.03322359  0.49582479  0.14842953  0.05853441]
[-0.02330709  0.29892085  0.14960022  0.39412096]
[-0.01732868  0.49163853  0.15748264  0.15209181]
[-0.00749591  0.68419609  0.16052447 -0.0870615 ]
[ 0.00618802  0.4871808   0.15878324  0.25165385]
[ 0.01593163  0.6797212   0.16381632  0.01296274]
[ 0.02952606  0.87216111  0.16407558 -0.2238857 ]
[ 0.04696928  1.06460428  0.15959786 -0.46065155]
[ 0.06826136  1.25715311  0.15038483 -0.69908138]
[ 0.09340443  1.06030152  0.1364032  -0.36308997]
[ 0.11461046  0.86353124  0.1291414  -0.0306978 ]
[ 0.13188108  0.66681657  0.12852745  0.29977921]


[ 0.06361027  0.59578985  0.00373929 -0.72402884]
[ 0.07552607  0.40061638 -0.01074129 -0.43017134]
[ 0.08353839  0.59588878 -0.01934471 -0.72622092]
[ 0.09545617  0.40103957 -0.03386913 -0.43968878]
[ 0.10347696  0.59662409 -0.04266291 -0.74285292]
[ 0.11540944  0.79230814 -0.05751997 -1.048651  ]
[ 0.13125561  0.98814416 -0.07849299 -1.35882076]
[ 0.15101849  0.79408935 -0.1056694  -1.09168804]
[ 0.16690028  0.60050654 -0.12750316 -0.83394249]
[ 0.17891041  0.79711842 -0.14418201 -1.16385077]
[ 0.19485277  0.60413735 -0.16745903 -0.91962558]
[ 0.20693552  0.41162617 -0.18585154 -0.68390141]
[ 0.21516805  0.21950433 -0.19952957 -0.45500771]
[ 0.21955813  0.02767997 -0.20862972 -0.2312531 ]
[ 0.02893066 -0.01340268  0.02558562  0.00354373]
[ 0.02866261  0.18134317  0.0256565  -0.28095809]
[ 0.03228947 -0.01413518  0.02003733  0.01970513]
[ 0.03200677  0.18069377  0.02043144 -0.26658905]
[ 0.03562065  0.37551825  0.01509966 -0.55275836]
[ 0.04313101  0.18018754  0.00404449 -0.25535656]


KeyboardInterrupt: 

### 4. Plot the Scores

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

### 5. Watch a Smart Agent!

In [ ]:
env = gym.make('CartPole-v0')

state = env.reset()
img = plt.imshow(env.render(mode='rgb_array'))
for t in range(1000):
    action, _ = policy.act(state)
    img.set_data(env.render(mode='rgb_array')) 
    plt.axis('off')
    display.display(plt.gcf())
    display.clear_output(wait=True)
    state, reward, done, _ = env.step(action)
    if done:
        break 

env.close()